In [1]:
# pip install pydantic

In [2]:
# pip install boto3

In [3]:
# pip install gspread

In [4]:
# pip install openai

In [20]:
import sys
import os


repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(repo_root)
print("Added to PYTHONPATH:", repo_root)

Added to PYTHONPATH: /home/jovyan/work


In [21]:
import json
import yaml
import boto3

from core.task_publisher import TaskPublisher
from core.process_engine import ProcessEngine
from core.process_registry import ProcessRegistry
from models.task import Task

In [22]:
tenant_id = "dev"

with open(f"../config/tenants/{tenant_id}.yaml", "r") as f:
    tenant_config = yaml.safe_load(f)

# DynamoDB
dynamodb_config = tenant_config.get("dynamodb", {})
dynamodb = boto3.resource("dynamodb", region_name=dynamodb_config.get("region"))
processes_table = dynamodb.Table(dynamodb_config.get("processes_table"))

# SQS
sqs_config = tenant_config.get("sqs", {})
sqs = boto3.client("sqs", region_name=sqs_config.get("region"))
task_publisher = TaskPublisher(sqs, sqs_config.get("tasks_url"))

print("Dynamo processes_table:", processes_table.table_name)
print("SQS tasks URL:", sqs_config.get("tasks_url"))

Dynamo processes_table: aie_agents_processes_dev
SQS tasks URL: https://sqs.us-east-2.amazonaws.com/355622872251/aie_agents_tasks_dev


In [4]:
event = "WHATSAPP_FILE_RECEIVED"
process_type = "CLIENT_BANK_STATEMENT_FOLLOW_UP"

context = {
    "client_id": 32423,
    "year": 2025,
    "month": 11,
    "file_id": "abc1234",
    "identity": "whatsapp:525571969848",
}

In [9]:
from core.processes.client_bank_statement import ClientBankStatementProcess

process_def = ProcessRegistry.get(process_type)
if process_def is None:
    raise RuntimeError(f"Unknown process_type in Task: {process_type}")

process_definitions = {
    process_type: process_def
}

In [10]:
engine = ProcessEngine(
    processes_table=processes_table,
    task_publisher=task_publisher,
    process_definitions=process_definitions,
)

result = engine.run(
    process_type=process_type,
    event=event,
    context=context,
)

print(result)

📤 Publishing task with delay=120s → PARSE_BANK_STATEMENT_PDF
{'process_key': 'alfa#CLIENT_BANK_STATEMENT_FOLLOW_UP#32423#2025#11', 'state': 'PENDING_REQUEST', 'tasks': [Task(task_type='PARSE_BANK_STATEMENT_PDF', agent_type='ACCOUNTING_JUNIOR', process_type='CLIENT_BANK_STATEMENT_FOLLOW_UP', context_key={'month': 11, 'year': 2025, 'client_id': 32423, 'identity': 'whatsapp:525571969848', 'file_id': 'abc1234'}, payload={'file_id': 'abc1234'}, debounce_policy={'type': 'messages_idle', 'min_idle_seconds': 120})]}


In [23]:
attrs = sqs.get_queue_attributes(
    QueueUrl=sqs_config.get("tasks_url"),
    AttributeNames=[
        "ApproximateNumberOfMessages",
        "ApproximateNumberOfMessagesNotVisible",
        "ApproximateNumberOfMessagesDelayed",
    ],
)
print(attrs["Attributes"])

ClientError: An error occurred (ExpiredToken) when calling the GetQueueAttributes operation: The security token included in the request is expired